In [27]:
import pandas as pd
dataset=pd.read_csv("CKD.csv")
#dataset

In [28]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype=int)
#dataset

In [29]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [30]:
dataset["classification_yes"].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [31]:
indep=dataset.drop(["classification_yes"],axis=1)
dep=dataset["classification_yes"]

In [32]:
indep.shape #find rows and cloumns

(399, 27)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [34]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [35]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'metric':['minkowski'],
             'n_neighbors':[5,7],'algorithm':['ball_tree','kd_tree','brute','auto']} 



grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute', 'auto'],
                         'metric': ['minkowski'], 'n_neighbors': [5, 7]},
             scoring='f1_weighted', verbose=3)

In [36]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002593,0.000489,0.013364,0.001850,ball_tree,minkowski,5,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.963284,0.962636,0.944023,0.962573,0.962573,0.959018,0.007502,1
1,0.002194,0.000399,0.014362,0.002570,ball_tree,minkowski,7,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.908877,0.944161,0.925524,0.981217,0.962573,0.944470,0.025706,5
2,0.004188,0.001163,0.012966,0.001669,kd_tree,minkowski,5,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.963284,0.962636,0.944023,0.962573,0.962573,0.959018,0.007502,1
3,0.004189,0.002631,0.014162,0.002475,kd_tree,minkowski,7,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.908877,0.944161,0.925524,0.981217,0.962573,0.944470,0.025706,5
4,0.001795,0.000746,0.225596,0.004942,brute,minkowski,5,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.963284,0.962636,0.944023,0.962573,0.962573,0.959018,0.007502,1
5,0.001795,0.000399,0.161967,0.076064,brute,minkowski,7,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.908877,0.944161,0.925524,0.981217,0.962573,0.944470,0.025706,5
6,0.001796,0.000399,0.070020,0.005723,auto,minkowski,5,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.963284,0.962636,0.944023,0.962573,0.962573,0.959018,0.007502,1
7,0.001197,0.000398,0.062650,0.004342,auto,minkowski,7,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.908877,0.944161,0.925524,0.981217,0.962573,0.944470,0.025706,5


In [37]:
grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
print("The confusion Matrix:\n",cm)
# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print("The report:\n",clf_report)

The confusion Matrix:
 [[51  0]
 [ 8 74]]
The report:
               precision    recall  f1-score   support

           0       0.86      1.00      0.93        51
           1       1.00      0.90      0.95        82

    accuracy                           0.94       133
   macro avg       0.93      0.95      0.94       133
weighted avg       0.95      0.94      0.94       133



In [38]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'algorithm': 'ball_tree', 'metric': 'minkowski', 'n_neighbors': 5}: 0.9404945931261721


In [41]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'algorithm': 'ball_tree', 'metric': 'minkowski', 'n_neighbors': 5}: 0.9404945931261721


In [42]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9992826398852224